This notebook uses the GPyTorch package to apply Gaussian Process regression to the multi-step energy consumption forecasting problem.

## Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gpytorch
import torch


from gpytorch.kernels import ScaleKernel, LinearKernel, PeriodicKernel, AdditiveKernel, ProductKernel
from gpytorch.constraints import Interval
from gpytorch.priors import NormalPrior
from gpytorch.variational import VariationalStrategy, CholeskyVariationalDistribution, NaturalVariationalDistribution
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import TensorDataset, DataLoader
from tqdm.notebook import tqdm

In [ ]:
random_seed = 1923

Using torch.float32 datatype seems to cause issues with zero values in matrix algebra. Using float64 and disabling mixed precision fixes them but slows down training quite a bit.

In [ ]:
# Set Torch settings
torch.set_default_dtype(torch.float64)
#torch.set_float32_matmul_precision('medium')

In [ ]:
# Plot settings
plt.rcParams["figure.autolayout"] = True
plt.rcParams['figure.dpi'] = 100
sns.set_style("darkgrid")

In [ ]:
output_dir = "./OutputData/"

In [ ]:
df = pd.read_csv(output_dir + "full_data.csv")
df["time"] = pd.to_datetime(df["time"], format = "%d:%m:%Y:%H:%M")

In [ ]:
# Drop generation columns
gen_cols = df.columns.values[2:].tolist()
df = df.drop(gen_cols, axis = 1)

In [ ]:
df

## Data prep

We do not need to cyclical encode seasonal features, as we will apply periodic kernels to them.

In [ ]:
# Add time columns

# Trend
df["trend"] = df.index.values

# Hour of day
df["hour"] = df.time.dt.hour + 1

# Day of week
df["dayofweek"] = df.time.dt.dayofweek + 1

# Month
df["month"] = df.time.dt.month

In [ ]:
df

In [ ]:
# Evaluation parameters that match the sequence2sequence testing scheme
horizon = 32 # Forecast horizon
first_t = df[df["time"] == '2022-10-18 16:00:00'].index[0] # First prediction point
stride = 24 # Number of timesteps between each prediction point

# Dataloader arguments
batch_size = 1024
shuffle = False
num_workers = 0

In [ ]:
# Split features & target
X = df.drop(["time", "consumption_MWh",], axis = 1).values
y = df["consumption_MWh"].values

In [ ]:
# Train-test split
X_train, X_test = X[:first_t, :], X[first_t:, :]
y_train, y_test = y[:first_t], y[first_t:]

In [ ]:
# Feature & target scaling (0-1), tensor conversion
scaler = MinMaxScaler()
X_train = torch.tensor(scaler.fit_transform(X_train))
X_test = torch.tensor(scaler.transform(X_test))

scaler_target = MinMaxScaler()
y_train = torch.tensor(scaler_target.fit_transform(y_train.reshape(-1, 1))).squeeze(-1)
y_test = torch.tensor(scaler_target.transform(y_test.reshape(-1, 1))).squeeze(-1)

In [ ]:
# Tensor datasets & dataloaders
train_data = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_data, batch_size = batch_size, num_workers = num_workers, shuffle = False)

test_data = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_data, batch_size = batch_size, num_workers = num_workers, shuffle = False)

## Model & wrapper definition

Summary so far:
- ExactGP can only be trained with unbatched gradient descent, roughly 10k observations. Does a good job for predicting the first few days of the testing set, declines to zero across time. Would likely be solved by online updates of training data.
- VariationalGP can be trained with with batched, stochastic & natural gradient descent. Only fits into GPU memory with few inducting points, because inducting points are unbatched, kind of defeating the purpose of using SGD. Does not fit & converge properly with few inducting points.
- VNNGP supports batching the inducting points, essentially using the entire data as both training & inducting points. Give it a try.

In [ ]:
#Variational GP model class
class GPModel(gpytorch.models.ApproximateGP):

    def __init__(self, inducing_points, learn_inducting_locations = False):

        # Initialize variational parameters
        variational_distribution = NaturalVariationalDistribution(inducing_points.size(0))
        variational_strategy = VariationalStrategy(
            self, inducing_points, variational_distribution, learn_inducing_locations = learn_inducting_locations)
        super(GPModel, self).__init__(variational_strategy)

        # Initialize mean function
        self.mean_module = gpytorch.means.ConstantMean()
        #self.mean_module = gpytorch.means.ZeroMean()

        # Initialize covariance kernel
        self.covariance_module = AdditiveKernel(
            LinearKernel(active_dims = 0),
            ScaleKernel(PeriodicKernel(
                active_dims = (1),
                period_length_prior = NormalPrior(24, 1) # Applied to hour feature
            )),
            ScaleKernel(PeriodicKernel(
                active_dims = (2),
                period_length_prior = NormalPrior(7, 1) # Applied to day of week feature
            )),
            ScaleKernel(PeriodicKernel(
                active_dims = (3),
                period_length_prior = NormalPrior(12, 1) # Applied to month feature
            )),
        )

    def forward(self, x):
        mean = self.mean_module(x)
        covar = self.covariance_module(x)
        return gpytorch.distributions.MultivariateNormal(mean, covar)

In [ ]:
# Variational GP wrapper class
class VariationalMinibatchGP:
    
    def __init__(self, model, likelihood, cuda = True):
        self.model = model
        self.likelihood = likelihood
        self.cuda = cuda

    # Training method
    def train(self, train_loader, num_data, max_epochs, learning_rate = 0.1, early_stop = 10, early_stop_tol = 1e-4):

        # Put model & likelihood on GPU if cuda is enabled
        if self.cuda:
            self.model = self.model.cuda()
            self.likelihood = self.likelihood.cuda()

        # Find optimal kernel hyperparameters
        self.model.train()
        self.likelihood.train()

        # Create variational optimizer for natural gradient descent
        var_optimizer = gpytorch.optim.NGD(
            self.model.variational_parameters(),
            num_data = num_data,
            lr = learning_rate
        )

        # Create hyperparameter optimizer with model parameters
        optimizer = torch.optim.Adam([
            {"params": self.model.parameters()},
            {"params": self.likelihood.parameters()}
        ], lr = learning_rate)

        # Create loss
        mll = gpytorch.mlls.VariationalELBO(self.likelihood, self.model, num_data = num_data)

        # N. of batches for epoch loss calculation
        num_batches = len(train_loader)

        # Training loop
        for epoch in range(max_epochs):

            # Initialize loss tracking for epoch
            total_loss_epoch = 0

            # Iterate over batches
            minibatch_iter = tqdm(train_loader, desc="Minibatch", leave=False)
            for X, y in minibatch_iter:

                # Put tensors on cuda if enabled
                if self.cuda:
                    X = X.cuda()
                    y = y.cuda()

                # Reset gradients
                optimizer.zero_grad()
                var_optimizer.zero_grad()
    
                # Get outputs from model
                output = self.model(X)
    
                # Calculate loss perform backpropagation, adjust weights
                batch_loss = -mll(output, y)
                batch_loss.backward()
                optimizer.step()
                var_optimizer.step()

                # Print batch loss
                batch_loss_scalar = batch_loss.item()
                minibatch_iter.set_postfix(loss = batch_loss_scalar)
    
                # Save batch loss
                total_loss_epoch += batch_loss_scalar

            # Calculate epoch loss
            epoch_loss = total_loss_epoch / num_batches
        
            # Initialize best loss & rounds with no improvement if first epoch
            if epoch == 0:
                self._best_epoch = epoch
                self._best_loss = epoch_loss
                self._epochs_no_improvement = 0
            
            # Record an epoch with no improvement
            if self._best_loss < epoch_loss - early_stop_tol:
                self._epochs_no_improvement += 1

            # Record an improvement in the loss
            if self._best_loss > epoch_loss:
                self.best_epoch = epoch
                self._best_loss = epoch_loss
                self._epochs_no_improvement = 0

            # Print epoch info
            print(f"Epoch complete: {epoch+1}/{max_epochs}, Loss: {epoch_loss}, Best loss: {self._best_loss}")

            # Early stop if necessary
            if self._epochs_no_improvement >= early_stop:
                print(f"Early stopping after epoch {epoch+1}")
                break

    # Method to update model training data (kernel hyperparameters unchanged, no additional training)
    # Returns an ExactGP model
    def update_train(self, X_update, y_update):
        
        # Put tensors on GPU if cuda is enabled
        if self.cuda:
            X_update = X_update.cuda()
            y_update = y_update.cuda()

        # Update model training data
        self.model = self.model.get_fantasy_model(X_update, y_update)

    # Predict method (unbatched)
    def predict(self, X_test, cpu = True, fast_preds = False):

        # Test data to GPU, if cuda enabled
        if self.cuda:
            X_test = X_test.cuda()

        # Activate eval mode
        self.model.eval()
        self.likelihood.eval()

        # Make predictions
        with torch.no_grad(), gpytorch.settings.fast_pred_var(state = fast_preds):

            # Returns the approximate model posterior distribution over functions p(f*|x*, X, y)
            # Noise is not yet added to the functions
            f_posterior = self.model(X_test)

            # Returns the approximate predictive posterior distribution p(y*|x*, X, y)
            # Noise is added to the functions
            y_posterior = self.likelihood(f_posterior)

            # Get posterior predictive mean & prediction intervals
            # By default, 2 standard deviations around the mean
            y_mean = y_posterior.mean
            y_lower, y_upper = y_posterior.confidence_region()

        # Return data to CPU if desired
        if cpu:
            y_mean = y_mean.cpu()
            y_lower = y_lower.cpu()
            y_upper = y_upper.cpu()

        return y_mean, y_lower, y_upper

## Model training & testing without online updates

In [ ]:
# Last N training points to use as inducing points
N_inducing_points = int(len(X_train) * 0.4)
#N_inducing_points = 24 * 365

In [ ]:
# Create model & likelihood
inducing_points = X_train.cuda()
#inducing_points = X_train[-N_inducing_points:, :].cuda() 
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = GPModel(inducing_points)
trainer = VariationalMinibatchGP(model, likelihood)

In [ ]:
# Perform training
trainer.train(
    train_loader,
    num_data = y_train.size(0), # N. of total observations in training data
    max_epochs = 50,
    early_stop = 5,
    early_stop_tol = 1e-3)

In [ ]:
# Get predictions
preds_mean, preds_upper, preds_lower = trainer.predict(X_test)

In [ ]:
# Plot predicted vs. actual, first 5 days of testing data
plt.plot(X_test[:120, 0], y_test[:120])
plt.plot(X_test[:120, 0], preds_mean[:120])
plt.fill_between(X_test[:120, 0], preds_lower[:120], preds_upper[:120], alpha = 0.2)

In [ ]:
# Plot predicted vs. actual, entire test set
plt.plot(X_test[:, 0], y_test)
plt.plot(X_test[:, 0], preds_mean)
plt.fill_between(X_test[:, 0], preds_lower, preds_upper, alpha = 0.2)

## Model testing with online updates

Use get_fantasy_model method to update trained model's training data with new input sequences. Hyperparameters are not updated, which kind of mirrors the usage of input sequnces in NN models.

In [ ]:
# Evaluation pseudocode:
Create preds list
Perform feature scaling
Train until [:first_t]
Predict on first_t + horizon
Save preds
For pred points in [first_t:] // stride:
    Perform feature scaling
    Expand training set & online train
    Predict on first_t + eval index * stride + horizon
    Save preds
Concat & return preds, actual targets[first_t:]

In [ ]:
# Plot predicted vs. actual, entire test set

In [ ]:
# Plot predicted vs. actual, select sequences

In [ ]:
# Calculate performance metrics